In [7]:
!pip install mlflow

In [ ]:
import os
import pandas as pd
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import mlflow
import mlflow.sklearn

# --- Configuration ---
RANDOM_STATE = 42
TEST_SIZE = 0.2
EXPERIMENT_NAME = "Regression_Model_Comparison_Demo"

def create_synthetic_data(n_samples=1000):
    """
    Creates a synthetic dataset for a regression task.
    Target (y) is linearly dependent on features with added noise.
    """
    print("--- 1. Data Creation ---")
    np.random.seed(RANDOM_STATE)

    # Generate features
    data = {
        'age': np.random.randint(20, 60, n_samples),
        'experience_years': np.random.rand(n_samples) * 10,
        'city_factor': np.random.choice([0.8, 1.0, 1.2], n_samples),
        'project_score': np.random.normal(loc=75, scale=15, size=n_samples)
    }

    df = pd.DataFrame(data)

    # Calculate continuous target variable (Salary Prediction)
    # y = 50000 + 1000*age + 5000*exp + 20000*city_factor + 100*score + noise
    df['salary_target'] = (
        50000 +
        df['age'] * 1000 +
        df['experience_years'] * 5000 +
        df['city_factor'] * 20000 +
        df['project_score'] * 100 +
        np.random.normal(0, 15000, n_samples) # Add significant noise
    )

    # Ensure target is positive
    df['salary_target'] = df['salary_target'].apply(lambda x: max(10000, x))

    print(f"✅ Created {n_samples} samples with 4 features and 'salary_target'.")
    return df

def perform_eda(df):
    """Prints basic EDA information."""
    print("\n--- 2. Exploratory Data Analysis (EDA) ---")

    # Print the first few rows
    print("\n[Data Head]:")
    print(df.head())

    # Print descriptive statistics
    print("\n[Descriptive Statistics]:")
    print(df.describe().T)

    # Print correlation with the target
    print("\n[Feature Correlation with Target]:")
    print(df.corr()['salary_target'].sort_values(ascending=False))

    # Note: In a real project, you would also create visualizations (histograms, scatter plots)
    print("💡 EDA complete. Features show clear correlation with the target.")

def preprocess_data(df):
    """
    Splits the data and performs standard scaling on features.
    Standard Scaling is essential for Linear Regression performance.
    """
    print("\n--- 3. Preprocessing and Splitting ---")

    # Define features and target
    X = df.drop('salary_target', axis=1)
    y = df['salary_target']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

    # Initialize and fit scaler on training features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print(f"✅ Data split into Train ({len(X_train)}) and Test ({len(X_test)}) sets.")
    print(f"✅ Features scaled using StandardScaler.")

    # Return features as DataFrames (for MLflow readability) and scaled numpy arrays (for model training)
    X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    return X_train_df, X_test_df, y_train, y_test, scaler

def evaluate_metrics(y_true, y_pred, prefix):
    """Calculates and returns standard regression metrics."""
    metrics = {
        f'{prefix}_mse': mean_squared_error(y_true, y_pred),
        f'{prefix}_mae': mean_absolute_error(y_true, y_pred),
        f'{prefix}_r2': r2_score(y_true, y_pred),
    }
    return metrics

def train_and_track_models(X_train, X_test, y_train, y_test):
    """
    Trains both models and uses MLflow to track parameters, metrics, and models.
    """
    print("\n--- 4. Training and MLflow Tracking ---")

    # Set up MLflow environment
    mlflow.set_experiment(EXPERIMENT_NAME)
    print(f"🔗 MLflow Experiment set to: '{EXPERIMENT_NAME}'")

    # ------------------------------------------------------------------
    # 4a. Linear Regression Training and Tracking
    # ------------------------------------------------------------------
    with mlflow.start_run(run_name="Linear_Regression_Model"):
        print("\n[MLflow Run 1: Linear Regression]")

        # 1. Define Model and Parameters
        model_lr = LinearRegression()

        # 2. Log Parameters
        mlflow.log_param("model_type", "LinearRegression")
        mlflow.log_param("fit_intercept", model_lr.get_params()['fit_intercept'])
        mlflow.log_param("scaler", "StandardScaler")
        mlflow.log_param("test_size", TEST_SIZE)

        # 3. Train Model (on scaled data)
        model_lr.fit(X_train, y_train)

        # 4. Predict
        y_pred_lr = model_lr.predict(X_test)

        # 5. Evaluate and Log Metrics
        metrics_lr = evaluate_metrics(y_test, y_pred_lr, "test")
        mlflow.log_metrics(metrics_lr)

        print(f"   Test MAE: ${metrics_lr['test_mae']:.2f}, R2: {metrics_lr['test_r2']:.4f}")

        # 6. Log Model (Artifact)
        mlflow.sklearn.log_model(model_lr, "Linear_Regression_Artifact")

        print(f"   ✅ Model and metrics logged to MLflow Run ID: {mlflow.active_run().info.run_id}")

    # ------------------------------------------------------------------
    # 4b. Random Forest Regressor Training and Tracking
    # ------------------------------------------------------------------
    with mlflow.start_run(run_name="Random_Forest_Model"):
        print("\n[MLflow Run 2: Random Forest Regressor]")

        # 1. Define Model and Parameters
        n_estimators = 200
        max_depth = 15

        model_rf = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=RANDOM_STATE,
            n_jobs=-1 # Use all cores
        )

        # 2. Log Parameters
        mlflow.log_param("model_type", "RandomForestRegressor")
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("test_size", TEST_SIZE)

        # 3. Train Model (Random Forest is less sensitive to scaling, but we use scaled data for consistency)
        model_rf.fit(X_train, y_train)

        # 4. Predict
        y_pred_rf = model_rf.predict(X_test)

        # 5. Evaluate and Log Metrics
        metrics_rf = evaluate_metrics(y_test, y_pred_rf, "test")
        mlflow.log_metrics(metrics_rf)

        print(f"   Test MAE: ${metrics_rf['test_mae']:.2f}, R2: {metrics_rf['test_r2']:.4f}")

        # 6. Log Model (Artifact)
        mlflow.sklearn.log_model(model_rf, "Random_Forest_Artifact")

        print(f"   ✅ Model and metrics logged to MLflow Run ID: {mlflow.active_run().info.run_id}")

    print("\n🏁 All experiments tracked successfully!")

    # Example prediction for comparison
    example_metrics = {
        "Linear Regression R2": metrics_lr['test_r2'],
        "Random Forest R2": metrics_rf['test_r2']
    }

    return model_lr, model_rf, example_metrics

def make_example_predictions(model_lr, model_rf, X_test, y_test, scaler):
    """Demonstrates how to use the models for prediction."""
    print("\n--- 5. Making Example Predictions ---")

    # Select the first test sample for prediction
    example_index = 0
    # Original (unscaled) features for display
    unscaled_features = X_test.iloc[example_index].to_dict()
    # Scaled features used for prediction
    scaled_features_array = X_test.iloc[example_index].values.reshape(1, -1)
    true_value = y_test.iloc[example_index]

    # Predict with Linear Regression
    pred_lr = model_lr.predict(scaled_features_array)[0]

    # Predict with Random Forest
    pred_rf = model_rf.predict(scaled_features_array)[0]

    print("\n[Input Data]:")
    print(json.dumps(unscaled_features, indent=4))
    print(f"\n[True Target Value]: ${true_value:.2f}")
    print("\n[Model Predictions]:")
    print(f"   Linear Regression: ${pred_lr:.2f}")
    print(f"   Random Forest Regressor: ${pred_rf:.2f}")
    print("---------------------------------")

def main():
    """Orchestrates the entire MLOps workflow."""

    # 1. Data Creation
    df = create_synthetic_data()

    # 2. EDA
    perform_eda(df)

    # 3. Preprocessing
    X_train_df, X_test_df, y_train, y_test, scaler = preprocess_data(df)

    # 4. Training and Tracking
    model_lr, model_rf, summary_metrics = train_and_track_models(
        X_train_df, X_test_df, y_train, y_test
    )

    # 5. Prediction Demo
    make_example_predictions(model_lr, model_rf, X_test_df, y_test, scaler)

    print("\n\n---------------------------------------------------")
    print("📋 Pipeline Complete!")
    print(f"Best Model (based on R2 score): {'Random Forest' if summary_metrics['Random Forest R2'] > summary_metrics['Linear Regression R2'] else 'Linear Regression'}")
    print("---------------------------------------------------")
    print("\n**NEXT STEP: View Experiment Tracking**")
    print("1. Open a new Command Prompt/Terminal.")
    print("2. Run the command: `mlflow ui`")
    print("3. Open your browser to http://127.0.0.1:5000 to see the detailed comparison of both models.")


if __name__ == "__main__":
    main()


2025/10/29 13:55:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


--- 1. Data Creation ---
✅ Created 1000 samples with 4 features and 'salary_target'.

--- 2. Exploratory Data Analysis (EDA) ---

[Data Head]:
   age  experience_years  city_factor  project_score  salary_target
0   58          1.042470          1.2      80.656506  146556.140638
1   48          8.404396          1.2      74.561058  126611.047815
2   34          9.106862          1.2      91.890754  159919.942917
3   27          1.228108          1.2      74.229094   90106.349851
4   40          2.359060          1.2      48.404513  148725.670776

[Descriptive Statistics]:
                   count           mean           std           min  \
age               1000.0      39.991000     11.780055     20.000000   
experience_years  1000.0       5.031007      2.844953      0.002375   
city_factor       1000.0       0.996800      0.164246      0.800000   
project_score     1000.0      75.222940     14.670964     29.707318   
salary_target     1000.0  142243.932839  23693.181328  68274.851360

2025/10/29 13:56:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


   ✅ Model and metrics logged to MLflow Run ID: 8ccfc75d41c84c9bbb22a24136eb5cc4

[MLflow Run 2: Random Forest Regressor]


In [ ]:
import os
import pandas as pd
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import mlflow
import mlflow.sklearn

# --- Configuration ---
RANDOM_STATE = 42
TEST_SIZE = 0.2
EXPERIMENT_NAME = "Regression_Model_Comparison_Demo"

def create_synthetic_data(n_samples=1000):
    """
    Creates a synthetic dataset for a regression task.
    Target (y) is linearly dependent on features with added noise.
    """
    print("--- 1. Data Creation ---")
    np.random.seed(RANDOM_STATE)

    # Generate features
    data = {
        'age': np.random.randint(20, 60, n_samples),
        'experience_years': np.random.rand(n_samples) * 10,
        'city_factor': np.random.choice([0.8, 1.0, 1.2], n_samples),
        'project_score': np.random.normal(loc=75, scale=15, size=n_samples)
    }

    df = pd.DataFrame(data)

    # Calculate continuous target variable (Salary Prediction)
    # y = 50000 + 1000*age + 5000*exp + 20000*city_factor + 100*score + noise
    df['salary_target'] = (
        50000 +
        df['age'] * 1000 +
        df['experience_years'] * 5000 +
        df['city_factor'] * 20000 +
        df['project_score'] * 100 +
        np.random.normal(0, 15000, n_samples) # Add significant noise
    )

    # Ensure target is positive
    df['salary_target'] = df['salary_target'].apply(lambda x: max(10000, x))

    print(f"✅ Created {n_samples} samples with 4 features and 'salary_target'.")
    return df

def perform_eda(df):
    """Prints basic EDA information."""
    print("\n--- 2. Exploratory Data Analysis (EDA) ---")

    # Print the first few rows
    print("\n[Data Head]:")
    print(df.head())

    # Print descriptive statistics
    print("\n[Descriptive Statistics]:")
    print(df.describe().T)

    # Print correlation with the target
    print("\n[Feature Correlation with Target]:")
    print(df.corr()['salary_target'].sort_values(ascending=False))

    # Note: In a real project, you would also create visualizations (histograms, scatter plots)
    print("💡 EDA complete. Features show clear correlation with the target.")

def preprocess_data(df):
    """
    Splits the data and performs standard scaling on features.
    Standard Scaling is essential for Linear Regression performance.
    """
    print("\n--- 3. Preprocessing and Splitting ---")

    # Define features and target
    X = df.drop('salary_target', axis=1)
    y = df['salary_target']

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )

    # Initialize and fit scaler on training features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print(f"✅ Data split into Train ({len(X_train)}) and Test ({len(X_test)}) sets.")
    print(f"✅ Features scaled using StandardScaler.")

    # Return features as DataFrames (for MLflow readability) and scaled numpy arrays (for model training)
    X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

    return X_train_df, X_test_df, y_train, y_test, scaler

def evaluate_metrics(y_true, y_pred, prefix):
    """Calculates and returns standard regression metrics."""
    metrics = {
        f'{prefix}_mse': mean_squared_error(y_true, y_pred),
        f'{prefix}_mae': mean_absolute_error(y_true, y_pred),
        f'{prefix}_r2': r2_score(y_true, y_pred),
    }
    return metrics

def train_and_track_models(X_train, X_test, y_train, y_test):
    """
    Trains both models and uses MLflow to track parameters, metrics, and models.
    """
    print("\n--- 4. Training and MLflow Tracking ---")

    # Set up MLflow environment
    mlflow.set_experiment(EXPERIMENT_NAME)
    print(f"🔗 MLflow Experiment set to: '{EXPERIMENT_NAME}'")

    # ------------------------------------------------------------------
    # 4a. Linear Regression Training and Tracking
    # ------------------------------------------------------------------
    with mlflow.start_run(run_name="Linear_Regression_Model"):
        print("\n[MLflow Run 1: Linear Regression]")

        # 1. Define Model and Parameters
        model_lr = LinearRegression()

        # 2. Log Parameters
        mlflow.log_param("model_type", "LinearRegression")
        mlflow.log_param("fit_intercept", model_lr.get_params()['fit_intercept'])
        mlflow.log_param("scaler", "StandardScaler")
        mlflow.log_param("test_size", TEST_SIZE)

        # 3. Train Model (on scaled data)
        model_lr.fit(X_train, y_train)

        # 4. Predict
        y_pred_lr = model_lr.predict(X_test)

        # 5. Evaluate and Log Metrics
        metrics_lr = evaluate_metrics(y_test, y_pred_lr, "test")
        mlflow.log_metrics(metrics_lr)

        print(f"   Test MAE: ${metrics_lr['test_mae']:.2f}, R2: {metrics_lr['test_r2']:.4f}")

        # 6. Log Model (Artifact)
        mlflow.sklearn.log_model(model_lr, "Linear_Regression_Artifact")

        print(f"   ✅ Model and metrics logged to MLflow Run ID: {mlflow.active_run().info.run_id}")

    # ------------------------------------------------------------------
    # 4b. Random Forest Regressor Training and Tracking
    # ------------------------------------------------------------------
    with mlflow.start_run(run_name="Random_Forest_Model"):
        print("\n[MLflow Run 2: Random Forest Regressor]")

        # 1. Define Model and Parameters
        n_estimators = 200
        max_depth = 15

        model_rf = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=RANDOM_STATE,
            n_jobs=-1 # Use all cores
        )

        # 2. Log Parameters
        mlflow.log_param("model_type", "RandomForestRegressor")
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("test_size", TEST_SIZE)

        # 3. Train Model (Random Forest is less sensitive to scaling, but we use scaled data for consistency)
        model_rf.fit(X_train, y_train)

        # 4. Predict
        y_pred_rf = model_rf.predict(X_test)

        # 5. Evaluate and Log Metrics
        metrics_rf = evaluate_metrics(y_test, y_pred_rf, "test")
        mlflow.log_metrics(metrics_rf)

        print(f"   Test MAE: ${metrics_rf['test_mae']:.2f}, R2: {metrics_rf['test_r2']:.4f}")

        # 6. Log Model (Artifact)
        mlflow.sklearn.log_model(model_rf, "Random_Forest_Artifact")

        print(f"   ✅ Model and metrics logged to MLflow Run ID: {mlflow.active_run().info.run_id}")

    print("\n🏁 All experiments tracked successfully!")

    # Example prediction for comparison
    example_metrics = {
        "Linear Regression R2": metrics_lr['test_r2'],
        "Random Forest R2": metrics_rf['test_r2']
    }

    return model_lr, model_rf, example_metrics

def make_example_predictions(model_lr, model_rf, X_test, y_test, scaler):
    """Demonstrates how to use the models for prediction."""
    print("\n--- 5. Making Example Predictions ---")

    # Select the first test sample for prediction
    example_index = 0
    # Original (unscaled) features for display
    unscaled_features = X_test.iloc[example_index].to_dict()
    # Scaled features used for prediction
    scaled_features_array = X_test.iloc[example_index].values.reshape(1, -1)
    true_value = y_test.iloc[example_index]

    # Predict with Linear Regression
    pred_lr = model_lr.predict(scaled_features_array)[0]

    # Predict with Random Forest
    pred_rf = model_rf.predict(scaled_features_array)[0]

    print("\n[Input Data]:")
    print(json.dumps(unscaled_features, indent=4))
    print(f"\n[True Target Value]: ${true_value:.2f}")
    print("\n[Model Predictions]:")
    print(f"   Linear Regression: ${pred_lr:.2f}")
    print(f"   Random Forest Regressor: ${pred_rf:.2f}")
    print("---------------------------------")

def main():
    """Orchestrates the entire MLOps workflow."""

    # 1. Data Creation
    df = create_synthetic_data()

    # 2. EDA
    perform_eda(df)

    # 3. Preprocessing
    X_train_df, X_test_df, y_train, y_test, scaler = preprocess_data(df)

    # 4. Training and Tracking
    model_lr, model_rf, summary_metrics = train_and_track_models(
        X_train_df, X_test_df, y_train, y_test
    )

    # 5. Prediction Demo
    make_example_predictions(model_lr, model_rf, X_test_df, y_test, scaler)

    print("\n\n---------------------------------------------------")
    print("📋 Pipeline Complete!")
    print(f"Best Model (based on R2 score): {'Random Forest' if summary_metrics['Random Forest R2'] > summary_metrics['Linear Regression R2'] else 'Linear Regression'}")
    print("---------------------------------------------------")
    print("\n**NEXT STEP: View Experiment Tracking**")
    print("1. Open a new Command Prompt/Terminal.")
    print("2. Run the command: `mlflow ui`")
    print("3. Open your browser to http://127.0.0.1:5000 to see the detailed comparison of both models.")


if __name__ == "__main__":
    main()